In [ ]:
#%pip install fairseq

In [42]:
import bark

In [26]:
from bark.generation import load_codec_model, generate_text_semantic
from encodec.utils import convert_audio

import torchaudio
import torch

model = load_codec_model(use_gpu=True)

In [22]:
# Load and pre-process the audio waveform
audio_filepath = 'christopher_lee.wav' # the audio you want to clone (will get truncated so 5-10 seconds is probably fine, existing samples that I checked are around 7 seconds)
device = 'cuda' # or 'cpu'
wav, sr = torchaudio.load(audio_filepath)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.unsqueeze(0).to(device)

In [23]:
# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

In [2]:
import os
import subprocess
import sys

# Unfortunately, fairseq kmeans package resolution is borked on my machine, so manually adding it
# TODO: Fix this
# Get the git repo root directory
git_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"]).strip().decode("utf-8")

# Append the desired subdirectory
feature_utils_path = os.path.join(git_root, "venv", "lib", "python3.10", "site-packages", "fairseq", "examples", "hubert", "simple_kmeans")

# Add the path to sys.path
sys.path.append(feature_utils_path)
from fairseq.examples.hubert.simple_kmeans.dump_hubert_feature import HubertFeatureReader


In [9]:
# TODO: DELETE THIS
from torch.hub import download_url_to_file

if not os.path.exists("models/hubert_base_ls960.pt"):
    # Yes, hard-coding the URL of the model is jank. Too bad!
    # Update this if this changes! https://github.com/facebookresearch/textlesslib/blob/698e6a039375bac0cd5f1b8683beeec5e8f702c0/textless/checkpoint_manager/__init__.py#L20
    download_url_to_file("https://dl.fbaipublicfiles.com/hubert/hubert_base_ls960.pt", "models/hubert_base_ls960.pt")

reader = HubertFeatureReader(
    ckpt_path=os.path.join("models", "hubert_base_ls960.pt"),
    layer=6
)

2023-05-11 20:59:06 | INFO | fairseq.tasks.hubert_pretraining | current directory is /home/ritsuko/projects/ai/audio/bark
2023-05-11 20:59:06 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/wnhsu/data/librispeech/960h/iter/250K_50hz_km100_mp0_65_v2', 'fine_tuning': False, 'labels': ['layer6.km500'], 'label_dir': None, 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2023-05-11 20:59:06 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0

In [24]:
semantic_hubert_feats = reader.get_feats(audio_filepath)


In [29]:
import torch
import torch.nn as nn

class hubert_to_wte_projection(nn.Module):
    def __init__(self, input_dim=768, output_dim=1024):
        super().__init__()
        self.proj = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.proj(x)

proj = hubert_to_wte_projection()
proj.to(device)
proj.load_state_dict(torch.load(os.path.join("models", "hubert_bark_proj_20230511-220924.pt")))

<All keys matched successfully>

In [33]:
with torch.no_grad():
    semantic_emb = proj(semantic_hubert_feats).cpu().numpy()

In [31]:
# move codes to cpu
codes = codes.cpu().numpy()

In [34]:
import numpy as np
voice_name = 'output' # whatever you want the name of the voice to be
output_path = 'bark/assets/prompts/' + voice_name + '.npz'
np.savez(output_path, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_emb)

In [ ]:
# That's it! Now you can head over to the generate.ipynb and use your voice_name for the 'history_prompt'

In [ ]:
# Heres the generation stuff copy-pasted for convenience

In [1]:
from bark.api import generate_audio
from transformers import BertTokenizer
from bark.generation import SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic

# Enter your prompt and speaker here
text_prompt = "Hello, my name is Serpy. And, uh — and I like pizza. [laughs]"
voice_name = "output" # use your custom voice name here if you have one

# load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

/home/ritsuko/projects/ai/audio/bark/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# download and load all models
preload_models(
    text_use_gpu=True,
    text_use_small=False,
    coarse_use_gpu=True,
    coarse_use_small=False,
    fine_use_gpu=True,
    fine_use_small=False,
    codec_use_gpu=True,
    force_reload=False,
    path="models"
)

In [3]:
# simple generation
audio_array = generate_audio(text_prompt, history_prompt=voice_name, text_temp=0.7, waveform_temp=0.7)

  2%|▏         | 2/100 [00:00<00:05, 17.09it/s]

Next:torch.Size([1, 1])
Out: torch.Size([1, 0])
X: torch.Size([1, 257, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 1])
X: torch.Size([1, 258, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 2])
X: torch.Size([1, 259, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 3])
X: torch.Size([1, 260, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 4])
X: torch.Size([1, 261, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 5])
X: torch.Size([1, 262, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 6])
X: torch.Size([1, 263, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 7])
X: torch.Size([1, 264, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 8])
X: torch.Size([1, 265, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 9])
X: torch.Size([1, 266, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 10])
X: torch.Size([1, 267, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 11])
X: torch.Size([1, 268, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 12])
X: torch.Size

  7%|▋         | 7/100 [00:00<00:04, 19.40it/s]

Next:torch.Size([1, 1])
Out: torch.Size([1, 30])
X: torch.Size([1, 287, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 31])
X: torch.Size([1, 288, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 32])
X: torch.Size([1, 289, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 33])
X: torch.Size([1, 290, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 34])
X: torch.Size([1, 291, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 35])
X: torch.Size([1, 292, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 36])
X: torch.Size([1, 293, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 37])
X: torch.Size([1, 294, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 38])
X: torch.Size([1, 295, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 39])
X: torch.Size([1, 296, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 40])
X: torch.Size([1, 297, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 41])
X: torch.Size([1, 298, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 42])
X: 

 10%|█         | 10/100 [00:00<00:04, 20.14it/s]

...

 60%|██████    | 60/100 [00:02<00:01, 20.36it/s]

Next:torch.Size([1, 1])
Out: torch.Size([1, 449])
X: torch.Size([1, 706, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 450])
X: torch.Size([1, 707, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 451])
X: torch.Size([1, 708, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 452])
X: torch.Size([1, 709, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 453])
X: torch.Size([1, 710, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 454])
X: torch.Size([1, 711, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 455])
X: torch.Size([1, 712, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 456])
X: torch.Size([1, 713, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 457])
X: torch.Size([1, 714, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 458])
X: torch.Size([1, 715, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 459])
X: torch.Size([1, 716, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 460])
X: torch.Size([1, 717, 1024])
Next:torch.Size([1, 1])
Out: torch.Size(

100%|██████████| 100/100 [00:03<00:00, 32.06it/s]


Next:torch.Size([1, 1])
Out: torch.Size([1, 480])
X: torch.Size([1, 737, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 481])
X: torch.Size([1, 738, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 482])
X: torch.Size([1, 739, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 483])
X: torch.Size([1, 740, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 484])
X: torch.Size([1, 741, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 485])
X: torch.Size([1, 742, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 486])
X: torch.Size([1, 743, 1024])
Next:torch.Size([1, 1])
Out: torch.Size([1, 487])
X: torch.Size([1, 744, 1024])


AssertionError: 

In [ ]:
# generation with more control
x_semantic = generate_text_semantic(
    text_prompt,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)

x_coarse_gen = generate_coarse(
    x_semantic,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)
x_fine_gen = generate_fine(
    x_coarse_gen,
    history_prompt=voice_name,
    temp=0.5,
)
audio_array = codec_decode(x_fine_gen)

In [ ]:
from IPython.display import Audio
# play audio
Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:
from scipy.io.wavfile import write as write_wav
# save audio
filepath = "/output/audio.wav" # change this to your desired output path
write_wav(filepath, SAMPLE_RATE, audio_array)